## Covid Predictions with ARIMA Model

In [1]:
#import the required packages 
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
import glob

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings('ignore')

from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
import pandas as pd
from pandas.tseries.offsets import *
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pmdarima import auto_arima
pd.TimeSeries = pd.Series

In [2]:
#Get the data
df = pd.read_csv(r'./train.csv', index_col=None, header=0)

In [3]:
#Get the required columns
dfConf = df[['Date','Province_State','Confirmed']]
dfDeath = df[['Date','Province_State','Deaths']]

In [4]:
#Check the data
print(dfConf.head())
print(dfDeath.head())

       Date Province_State  Confirmed
0  8/1/2020        Alabama    89349.0
1  8/1/2020         Alaska     3136.0
2  8/1/2020        Arizona   177002.0
3  8/1/2020       Arkansas    43173.0
4  8/1/2020     California   506613.0
       Date Province_State  Deaths
0  8/1/2020        Alabama  1603.0
1  8/1/2020         Alaska    24.0
2  8/1/2020        Arizona  3747.0
3  8/1/2020       Arkansas   458.0
4  8/1/2020     California  9345.0


In [5]:
#Remove the hours from the time column:
dfConf['Date'] = pd.to_datetime(dfConf['Date']).dt.date
dfDeath['Date'] = pd.to_datetime(dfDeath['Date']).dt.date

### Required Functions for ARIMA and ARMA

In [6]:
# A function for forcasting and returning the new table
# This only work for the next 26 days
#ARMA model
def ARMA_MODEL_FORECAST_Confirmed(ds):
    
    #Get the confimred values of the state
    ConfValue = ds.Confirmed.values
    
    #Find the time series
    timeser = pd.TimeSeries(ConfValue , index=pd.to_datetime(ds.Date))
    
    #It is a non-stationary graph so we can turn it into stationary by taking the log
    ts_log_data = np.log(timeser)
    try:
        #Use Autoregressive–moving-average model for prediction
        model = sm.tsa.ARMA(ts_log_data, order=(3, 0)).fit()
        
        #Set the starting and Ending dates
        start_date = ts_log_data.index[-1] + Day(1)
        end_date = ts_log_data.index[-1] + Day(26)
        
        #Predict the values
        y_forecast = model.predict(start_date.isoformat(), end_date.isoformat())
        
        #Return a table with the new values
        Final_Table = pd.Series(np.exp(y_forecast).values, np.arange(1, 27))
        
        return Final_Table
    
    #if there is a convergance error pass it
    except Exception:
        pass

    
#ARIMA model
def ARIMA_MODEL_FORECAST_Confirmed(ds):
    
    #Get the confimred values of the state
    ConfValue = ds.Confirmed.values
    
    #Find the time series
    timeser = pd.TimeSeries(ConfValue , index=pd.to_datetime(ds.Date))
    
    try:
        #Use Autoregressive integrated moving average model for prediction
        #Trend is not constant
        model = sm.tsa.ARIMA(timeser,order=(0,2,1)).fit(transparams=False, trend='nc')
        
        #Set the starting and Ending dates
        start_date = timeser.index[-1] + Day(1)
        end_date = timeser.index[-1] + Day(26)
        
        #Predict the values
        y_forecast = model.predict(start_date.isoformat(),end_date.isoformat(),typ='levels')

        
        #Return a table with the new values
        Final_Table = pd.Series(y_forecast.values, np.arange(1, 27))
        
        return Final_Table
    
    #if there is a convergance error pass it
    except Exception:
        pass

In [7]:
# A function for forcasting and returning the new table
# This only work for the next 26 days
#ARMA model
def ARMA_MODEL_FORECAST_DEATHS(ds):
    
    #Get the confimred values of the state
    DeathValue = ds.Deaths.values
    
    #Find the time series
    timeser = pd.TimeSeries(DeathValue , index=pd.to_datetime(ds.Date))
    
    #It is a non-stationary graph so we can turn it into stationary by taking the log
    ts_log_data = np.log(timeser)
    try:
        #Use Autoregressive–moving-average model for prediction
        model = sm.tsa.ARMA(ts_log_data, order=(3, 0)).fit()
        
        #Set the starting and Ending dates
        start_date = ts_log_data.index[-1] + Day(1)
        end_date = ts_log_data.index[-1] + Day(26)
        
        #Predict the values
        y_forecast = model.predict(start_date.isoformat(), end_date.isoformat())
        
        #Return a table with the new values
        Final_Table = pd.Series(np.exp(y_forecast).values, np.arange(1, 27))
        
        return Final_Table
    
    #if there is a convergance error pass it
    except Exception:
        pass

#ARIMA model    
def ARIMA_MODEL_FORECAST_DEATHS(ds):
    
    #Get the confimred values of the state
    DeathValue = ds.Deaths.values
    
    #Find the time series
    timeser = pd.TimeSeries(DeathValue , index=pd.to_datetime(ds.Date))
    
    try:       
        #Use Autoregressive integrated moving average model for prediction
        model = sm.tsa.ARIMA(timeser,order=(0,2,1)).fit()
        
        
        #Set the starting and Ending dates
        start_date = timeser.index[-1] + Day(1)
        end_date = timeser.index[-1] + Day(26)
        
        #Predict the values
        y_forecast = model.predict(start_date.isoformat(),end_date.isoformat(),typ='levels')

        
        #Return a table with the new values
        Final_Table = pd.Series(y_forecast.values, np.arange(1, 27))
        
        return Final_Table
    
    #if there is a convergance error pass it
    except Exception:
        pass

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
GroupByStateCases = dfConf.groupby('Province_State')
FinalTableCases = GroupByStateCases.apply(ARIMA_MODEL_FORECAST_Confirmed)

## Number of Cases for the next 26 days:

In [10]:
FinalTableCases

,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,21,22,23,24,25,26
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,127306.299499,128554.598999,129802.898498,131051.197997,132299.497496,133547.796996,134796.096495,136044.395994,137292.695493,138540.994993,...,147279.091487,148527.390987,149775.690486,151023.989985,152272.289484,153520.588984,154768.888483,156017.187982,157265.487481,158513.786981
Alaska,5335.068966,5407.137932,5479.206897,5551.275863,5623.344829,5695.413795,5767.482760,5839.551726,5911.620692,5983.689658,...,6488.172418,6560.241384,6632.310349,6704.379315,6776.448281,6848.517247,6920.586213,6992.655178,7064.724144,7136.793110
Arizona,202239.044438,202643.088875,203047.133313,203451.177751,203855.222189,204259.266626,204663.311064,205067.355502,205471.399939,205875.444377,...,208703.755441,209107.799879,209511.844316,209915.888754,210319.933192,210723.977629,211128.022067,211532.066505,211936.110943,212340.155380
Arkansas,61894.437370,62564.874740,63235.312111,63905.749481,64576.186851,65246.624221,65917.061592,66587.498962,67257.936332,67928.373702,...,72621.435294,73291.872664,73962.310034,74632.747404,75303.184775,75973.622145,76644.059515,77314.496885,77984.934256,78655.371626
California,718278.608313,724082.216626,729885.824938,735689.433251,741493.041564,747296.649877,753100.258189,758903.866502,764707.474815,770511.083128,...,811136.341317,816939.949630,822743.557943,828547.166255,834350.774568,840154.382881,845957.991194,851761.599506,857565.207819,863368.816132
Colorado,57689.121277,57974.242555,58259.363832,58544.485110,58829.606387,59114.727665,59399.848942,59684.970219,59970.091497,60255.212774,...,62251.061716,62536.182994,62821.304271,63106.425549,63391.546826,63676.668103,63961.789381,64246.910658,64532.031936,64817.153213
Connecticut,52971.586227,53064.172455,53156.758682,53249.344909,53341.931137,53434.517364,53527.103591,53619.689819,53712.276046,53804.862274,...,54452.965865,54545.552092,54638.138320,54730.724547,54823.310774,54915.897002,55008.483229,55101.069456,55193.655684,55286.241911
Delaware,17514.034483,17599.068966,17684.103448,17769.137931,17854.172414,17939.206897,18024.241380,18109.275863,18194.310345,18279.344828,...,18874.586208,18959.620691,19044.655174,19129.689656,19214.724139,19299.758622,19384.793105,19469.827588,19554.862070,19639.896553
Florida,625319.514265,627168.028530,629016.542796,630865.057061,632713.571326,634562.085591,636410.599856,638259.114121,640107.628387,641956.142652,...,654895.742508,656744.256773,658592.771038,660441.285304,662289.799569,664138.313834,665986.828099,667835.342364,669683.856629,671532.370895


In [11]:
GroupByStateDeaths = dfDeath.groupby('Province_State')
FinalTableDeaths = GroupByStateDeaths.apply(ARIMA_MODEL_FORECAST_DEATHS)

## Number of Deaths for the next 26 days:

In [12]:
FinalTableDeaths

,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,21,22,23,24,25,26
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,2199.741713,2217.387543,2234.937488,2252.391548,2269.749724,2287.012016,2304.178424,2321.248948,2338.223587,2355.102342,...,2470.568865,2486.680546,2502.696342,2518.616254,2534.440282,2550.168425,2565.800684,2581.337059,2596.777550,2612.122156
Alaska,37.686763,38.388431,39.105005,39.836485,40.582870,41.344160,42.120356,42.911457,43.717463,44.538375,...,50.702112,51.642268,52.597329,53.567295,54.552167,55.551945,56.566628,57.596216,58.640710,59.700109
Arizona,5059.601011,5089.349634,5118.245871,5146.289721,5173.481184,5199.820260,5225.306949,5249.941251,5273.723166,5296.652694,...,5433.292557,5449.402990,5464.661035,5479.066694,5492.619966,5505.320851,5517.169349,5528.165460,5538.309184,5547.600521
Arkansas,810.921691,825.010901,839.267630,853.691879,868.283648,883.042935,897.969743,913.064070,928.325916,943.755282,...,1056.451387,1073.220908,1090.157949,1107.262510,1124.534589,1141.974189,1159.581307,1177.355946,1195.298103,1213.407780
California,13131.134779,13239.349229,13346.643350,13453.017143,13558.470607,13663.003743,13766.616550,13869.309028,13971.081178,14071.933000,...,14752.126549,14845.615742,14938.184606,15029.833141,15120.561348,15210.369227,15299.256776,15387.223998,15474.270890,15560.397454
Colorado,1949.002168,1953.043268,1957.123300,1961.242265,1965.400161,1969.596990,1973.832751,1978.107444,1982.421069,1986.773626,...,2018.331627,2022.995641,2027.698588,2032.440466,2037.221277,2042.041020,2046.899695,2051.797302,2056.733842,2061.709313
Connecticut,4465.450910,4465.858884,4466.223921,4466.546021,4466.825185,4467.061413,4467.254703,4467.405058,4467.512475,4467.576956,...,4466.826099,4466.547087,4466.225139,4465.860254,4465.452433,4465.001675,4464.507980,4463.971349,4463.391781,4462.769277
Delaware,605.697886,606.398443,607.101668,607.807564,608.516129,609.227363,609.941267,610.657841,611.377085,612.098998,...,617.227139,617.970409,618.716349,619.464958,620.216237,620.970186,621.726804,622.486092,623.248050,624.012677
Florida,11289.413125,11389.397546,11486.953264,11582.080278,11674.778588,11765.048194,11852.889097,11938.301296,12021.284791,12101.839583,...,12597.719421,12658.844583,12717.541042,12773.808796,12827.647847,12879.058195,12928.039838,12974.592778,13018.717014,13060.412547


In [13]:
#changing the shape of the table
part1C = FinalTableCases[1]

for i in range(26):
    if (i + 2 < 27):
        part2C = FinalTableCases[i+2]
        part1C = pd.concat([part1C, part2C], ignore_index=True)

In [14]:
#changing the shape of the table
part1D = FinalTableDeaths[1]

for i in range(26):
    if (i + 2 < 27):
        part2D = FinalTableDeaths[i+2]
        part1D = pd.concat([part1D, part2D], ignore_index=True)

In [15]:
#Round the numbers amd get the final table
data = {'ForecastID': range(0, 1300 ,1), 
        'Confirmed':part1C, 
        'Deaths':part1D}
  
# Create DataFrame 
table = pd.DataFrame(data)
table['Confirmed'] = table['Confirmed'].round(decimals=0)
table['Deaths'] = table['Deaths'].round(decimals=0)

table

,ForecastID,Confirmed,Deaths
0,0,127306.0,2200.0
1,1,5335.0,38.0
2,2,202239.0,5060.0
3,3,61894.0,811.0
4,4,718279.0,13131.0
...,...,...,...
1295,1295,146426.0,2941.0
1296,1296,86873.0,2026.0
1297,1297,13271.0,355.0
1298,1298,94731.0,1246.0


In [16]:
#get the csv file
table.to_csv(r'./Team27.csv', index = False)